# Summary

We will look at helical coordinates and the steps required to set up and utilize them. You still start by creating helical filters of different sizes and dimensions and test the convolution and deconvolution with these filters. Then you will precondition an optimization problem using helical deconvolution

# Introduction



In the textbook we saw how helical filters can handle any number of dimensions efficiently and easily. Moreover, they allow several operations, such as deconvolution and factorization, in any dimensions using corresponding 1D operators. This makes helical filters and operators very useful for many applications, especially preconditioning.

Preconditioning can provide great speed up and efficiency improvements for optimization. However, it requires knowing the inverse of the regularization operator. In many cases, it is very difficult to estimate the inverse operator. However, helical deconvolution allows us to divide by the filter response of the operator, assuming stationarity, without having to know the inverse.

# Your assignment

In this lab, you will explore and utilize the python helix classes that we wrote in class a few weeks ago. We encourage you to use any dataset that is in the ``data`` directory found within your ``GP211`` directory or any previous datasets that we have used thus far in the labs. You can use the other labs to help you read in the datasets. Also, the terminal option within the jupyter notebook is a nice way to quickly explore the directory structure and open files with standard Unix text editors such as ``vim`` and ``nano``.

As part of your assignment, you will first creat 2D helix filters and test their convolution and deconvolution operators. Then, you will run and compare a regularized optimization to a preconditioned optimization. We will only be providing you with this helix filter codes that we created in class.

### Helix filter code

Please find the codes for HelixFilter.py, Helicon.py and Polydiv.py that we implemented together in class.

#### Helix2Cart

In [1]:
import giee
import copy

class helix2cart:
    """Convert to and from cartesian and helix space"""
    def __init__(self,nd):
        """Initialize conversion
        
             nd - Data dimensions"""
        if not isinstance(nd,list):
            raise Exception("Expecting nd to be a list")
    
        self._ndim=copy.deepcopy(nd)
        self._b=[1]
        sz=1
        for n in self._ndim:
            if not isinstance(n,int):
                raise Exception("Expecting a list of ints")
            sz=sz*n
            self._b.append(sz)
    

    def toCart(self,hlx):
        cart=[0]*len(self._ndim)
        lft=hlx
        for i in range(len(self._ndim)-1,-1,-1):
            cart[i]=int(lft/self._b[i])
            lft-=cart[i]*self._b[i]
        return cart
    
    
    def toHelix(self,cart):
        """Convert from cartesian space to helix space"""
        if len(cart) != len(self._ndim):
            raise Exception("Expecting cart to be same size as data")
        hlx=0
        for i in range(len(self._ndim)):
            if not isinstance(cart[i],int):
                raise Exception("Expecting cart to be a list of ints")
            hlx+=cart[i]*self._b[i]
        return hlx 

WARNING! DATAPATH not found. The folder /tmp will be used to write binary files


#### HelixFilter

In [2]:
import Helix2cart
import giee

class helixFilter(giee.vector):
    """Class for defining a filter on a helix"""
    def __init__(self,nd,**kw):
        """Option 1:
           nelem [int] Number of elements in filter
        
           Option 2:
            filt -  Filter to make a copy of
            
            Option 3:
             n - Length of a box describing the filter. First axis must be odd
        
        """
        self._nd=nd

        if "nelem" in kw: 
            if not isinstance(kw["nelem"],int):
                raise Exception("Expecting nelem to be integer")
            super().__init__([kw["nelem"]])
            self._lags=[0]*self.getNdArray().shape[0]

        elif "filt" in kw: 
            if not isinstance(kw["filt"],helixFilter):
                raise Exception("Expecting filter to be helixFilter")
            super().__init__(kw["filt"]._hyper,arr=kw["filt"].getNdArray())
            self._lags=kw["filt"]._lags

        elif "n" in kw: 
            self._lags=[]
            if not isinstance(kw["n"],list):
                raise Exception("Expecting n to be a list")
  
            if len(kw["n"]) > len(nd):
                raise Exception("Box dimensions larger than data")
            if len(kw["n"]) >3 :
                raise Exception("Can only handle 3-D")
            n=kw["n"]
            for i in range (len(nd),3):
                nd.append(1)
            for i in range(len(n),3):
                n.append(1)
            for i in range(3):
                if not isinstance(nd[i],int):
                    raise Exception("Expecting nd elements to be int")
                if not isinstance(n[i],int):
                    raise Exception("Expecting n elements to be int") 
                if n[i]> nd[i]:
                    raise Exception("Expecting n to be smaller than nd")


#### Helicon

In [3]:
import pyOperator
import HelixFilter
from numba import jit 
import numpy as np


class helicon(pyOperator.Operator):
    """ Filtering with the helix"""
    def __init__(self,model,data,filt):
        """ 
            model - vector with hypercube and ndArray
            data  - vector with hypercube and ndArray
            filt  - HelixFilter
        
        """
    
        if not model.checkSame(data):
            raise Exception("Model and data must be same space")
    
        try:
            h  =  model.getHyper()
            m  =  model.getNdArray()
            h2 = data.getHyper()
            d  = data.getNdArray()
        except:
            raise Exception("Model must have a hypercube and numpy representation")
    
        if not isinstance(filt,HelixFilter.helixFilter):
            raise Exception("Expecting filt to be a helix filter")
    
        ns=h.getNs()
        self._n123=h.getN123()

        if len(ns) !=len(filt._nd):
            for i in range(len(ns),len(filt._nd)):
                if filt._nd[i] >1: 
                    raise Exception("Expecting filter n to be the same as data")
    
        for i in range(len(ns)):
            if ns[i] != filt._nd[i]:
                raise Exception("Expecting filter n to be the same ss data")

        super().__init__(model,data)
        self._filt=filt
    
    def forward(self,add,model,data):
        """Forward helix filtering"""
        self.checkDomainRange(model,data)
        if not add:
            data.zero()
        m=np.reshape(model.getNdArray(),(self._n123,))
        d=np.reshape(data.getNdArray(),(self._n123,))
        heliconFor(m,d,self._filt._lags,self._filt.getNdArray())
        
    def adjoint(self,add,model,data):
        """Forward helix filtering"""
        self.checkDomainRange(model,data)
        if not add:
            model.zero()
        m=np.reshape(model.getNdArray(),(self._n123,))
        d=np.reshape(data.getNdArray(),(self._n123,))
        heliconAdj(m,d,self._filt._lags,self._filt.getNdArray())

@jit()
def heliconFor(model,data,lags,coefs):

    for i in range(model.shape[0]):
        data[i]+=model[i]
        for ilag in range(len(lags)):
            im=i-lags[ilag]
            if im>=0:
                data[i]+=model[im]*coefs[ilag]

@jit()
def heliconAdj(model,data,lags,coefs):
    for i in range(model.shape[0]):
        model[i]+=data[i]
        for ilag in range(len(lags)):
            im=i-lags[ilag]
            if im>=0:
                model[im]+=data[i]*coefs[ilag]


#### Polydiv

In [4]:
import pyOperator
import HelixFilter
import numpy as np

from numba import jit 
class polydiv(pyOperator.Operator):
    """ Inverse filtering with the helix"""
    def __init__(self,model,data,filt):
        """ 
            model - vector with hypercube and ndArray
            data  - vector with hypercube and ndArray
            filt  - HelixFilter
        
        """
    
        if not model.checkSame(data):
            raise Exception("Model and data must be same space")
    
        try:
            h=model.getHyper()
            m=model.getNdArray()
            h2=data.getHyper()
            d=data.getNdArray()
        except:
            raise Exception("Model must have a hypercube and numpy representation")
    
        if not isinstance(filt,HelixFilter.helixFilter):
            raise Exception("Expecting filt to be a helix filter")
    
        ns=h.getNs()
        self._n123=h.getN123()
        if len(ns) !=len(filt._nd):
            for i in range(len(ns),len(filt._nd)):
                if filt._nd[i] >1: 
                    raise Exception("Expecting filter n to be the same as data")
    
        for i in range(len(ns)):
            if ns[i] != filt._nd[i]:
                raise Exception("Expecting filter n to be the same ss data")

        super().__init__(model,data)
        self._filt=filt
        self._tt=model.clone()
        self._t=np.reshape(self._tt.getNdArray(),(self._n123,))

    def forward(self,add,model,data):
        """Forward helix filtering"""
        self.checkDomainRange(model,data)

        if not add:
            data.zero()
        self._tt.scaleAdd(model,0.,1.)
        m=np.reshape(model.getNdArray(),(self._n123,))
        d=np.reshape(data.getNdArray(),(self._n123,))
        polydivFor(m,d,\
                   self._filt._lags,self._filt.getNdArray(),self._t)
        data.scaleAdd(self._tt)

    def adjoint(self,add,model,data):
        """Forward helix filtering"""
        self.checkDomainRange(model,data)
        if not add:
            model.zero()
        self._tt.scaleAdd(data,0.,1.)
        m=np.reshape(model.getNdArray(),(self._n123,))
        d=np.reshape(data.getNdArray(),(self._n123,))

        polydivAdj(m,d,\
                   self._filt._lags,self._filt.getNdArray(),self._t)
        model.scaleAdd(self._tt)


@jit()
def polydivFor(model,data,lags,coefs,tt):

    for i in range(model.shape[0]):
        for ilag in range(len(lags)):
            im=i-lags[ilag]
            if im>=0:
                tt[i] -= tt[im]*coefs[ilag]
@jit()
def polydivAdj(model,data,lags,coefs,tt):
    for i in range(model.shape[0]-1,-1,-1):
        for ilag in range(len(lags)):
            im=i+lags[ilag]
            if im<model.shape[0]:
                tt[i]-=tt[im]*coefs[ilag]


### Helix Filters

1. Create and plot a 2D model with some spares spikes and make sure some spikes are close to the edges and cordners. Create a helix filter of your chocice (be creative) for the spiky model. Apply and plot the forward convolution of the filter on the model to see the filter response. Is this response what you expect? How does the filter behave around the edges?

2. Apply and plot the adjoint convolution on the result of the previous question. Is the adjoint a good approximation to the inverse? Why are the results symmetric at each spike location?


3. Apply and plot the forward deconvolution of the filter on the model to see the inverse filter response. How does the inverse response compare to the filter itself (please be elaborate)?


4. Apply and plot the adjoint deconvolution on the result of the previous question. How does this result compare to the results in the second question (again, please be elaborate)?


5. Use the deconvolution operator on the results of the second question to remove the effect of the adjoint convolution operator. Did the deconvolution operator correctly remove the adjoint convolution effects?

6. Use the deconvolution operator on the results of the previous question to remove the effect of the forward convolution operator. Did the deconvolution operator recover the original spiky model? Plot your results.

### Preconditioning

7. Set up a regularized optimization of your choice. You can either pick a problem done in a previous lab or make a new one given the datasets and operators available to you. The problem you pick should require regularization to give proper results where the regularization operator is stationary but not only diagonal. Clearly define the objective function and all of its components.

8. Why does this problem need to be regularized?

9.  Run the regularized solver and plot the original data, the inverted model, the reconstructed data and the curve of the residual norm as a function of iterations (might be good to show the logarithm of this curve normalized by its maximum value). How well did the optimization solve the problem?

10. Rewrite the objective function after preconditioning and clearly define all of its components


11. Convert your regularization operator into a helix filter and run the preconditioned solver with the deconvolution operator as the inverse of the regularization operator. Plot the original data, the inverted model, the reconstructed data and the curve of the residual norm as a function of iterations. How does the preconditioned solver compare to the regularized solver in both accuracy and convergence rate?

### Extra credit

1. If the regularization operator is non-stationary or non-invertible, can we do an approximate preconditioning? What would the objective function be in that case?